
1. 1초 단위 전처리
2. 1% 단위로 데이터 행 추출 (100개 세트 or 이하)
3. 각 세트별 행 개수 구하기 (1%감소까지 걸린 초)
4. 초를 기준으로 나쁨 보통 좋음으로 분류
5. 각 분류에서 높은 수치, 낮은 수치확인
6. x1,x2,x3.... 값 선정..

In [1]:
import scipy.io
import csv
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, chardet, glob
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from keras import callbacks
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

#TripA 타임 1초단위로 합하기
for i in range(32):
    globals()[i+1]=pd.read_csv('./TripA/TripA{}.csv'.format(i+1),sep=';')
    globals()[i+1] = globals()[i+1].iloc[::10,:]
    globals()[i+1].to_csv('./NewTripA/TripA{}.csv'.format(i+1),index=False)

#TripB 타임 1초 단위로 합하기
for i in range(38):
    globals()[i+1]=pd.read_csv('./TripB/TripB{}.csv'.format(i+1),sep=';')
    globals()[i+1] = globals()[i+1].iloc[::10,:]
    globals()[i+1].to_csv('./NewTripB/TripB{}.csv'.format(i+1),index=False)

In [ ]:
#TripA SoC 1퍼 단위로 각각 가져오기 및 저장
for j in range(32):
    globals()[j+1] = pd.read_csv('./NewTripA/TripA{}.csv'.format(j+1))
    startval = 0
    endval = 1
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            globals()[i].to_csv(f'./TripAsoc/TripA{j+1}_soc{i}.csv',index=False)
        startval += 1
        endval += 1

In [ ]:
#TripB SoC 1퍼 단위로 각각 가져오기 및 저장
for j in range(38):
    globals()[j+1] = pd.read_csv('./NewTripB/TripB{}.csv'.format(j+1))
    startval = 0
    endval = 1
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            globals()[i].to_csv(f'./TripBsoc/TripB{j+1}_soc{i}.csv',index=False)
        startval += 1
        endval += 1

globals()[1].shape[0]

In [10]:
#TripA SoC 1%감소 데이터셋 명 : 행 개수로 딕셔너리 만들기
TripA_SoC_len = {}

for j in range(32):
    globals()[j+1] = pd.read_csv('./NewTripA/TripA{}.csv'.format(j+1))
    startval = 0
    endval = 1
    # globals()['A_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            TripA_SoC_len ['TripA{}'.format(j+1)+' '+str(i)] = globals()[i].shape[0]
        startval += 1
        endval += 1


In [68]:
len(TripA_SoC_len)

360

In [11]:
#TripB SoC 1% 감소 데이터셋 명 : 행 개수로 딕셔너리 만들기

TripB_SoC_len = {}

for j in range(38):
    globals()[j+1] = pd.read_csv('./NewTripB/TripB{}.csv'.format(j+1))
    startval = 0
    endval = 1
    # globals()['B_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            TripB_SoC_len ['TripB{}'.format(j+1)+' '+str(i)] = globals()[i].shape[0]
        startval += 1
        endval += 1

In [76]:
len(TripB_SoC_len)

699

In [12]:
# TripA SoC 1% 감소 행 개수 빠름 보통 좋음 으로 분류
TripA_SoC_len_classification = {}

for j in range(32):
    globals()[j+1] = pd.read_csv('./NewTripA/TripA{}.csv'.format(j+1))
    startval = 0
    endval = 1
    # globals()['B_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            if globals()[i].shape[0] >= 150:
                TripA_SoC_len_classification ['TripA{}'.format(j+1)+' '+str(i)] = '좋음'
            elif globals()[i].shape[0] <50:
                TripA_SoC_len_classification ['TripA{}'.format(j+1)+' '+str(i)] = '빠름'
            else:
                TripA_SoC_len_classification ['TripA{}'.format(j+1)+' '+str(i)] = '보통'
        startval += 1
        endval += 1

In [72]:
TripA_SoC_len_classification

{'TripA1 81': '보통',
 'TripA1 82': '좋음',
 'TripA1 83': '좋음',
 'TripA1 84': '좋음',
 'TripA1 85': '좋음',
 'TripA1 86': '좋음',
 'TripA2 66': '빠름',
 'TripA2 67': '보통',
 'TripA2 68': '보통',
 'TripA2 69': '보통',
 'TripA2 70': '좋음',
 'TripA2 71': '좋음',
 'TripA2 72': '보통',
 'TripA2 73': '좋음',
 'TripA2 74': '보통',
 'TripA2 75': '빠름',
 'TripA2 76': '보통',
 'TripA2 77': '보통',
 'TripA2 78': '보통',
 'TripA2 79': '빠름',
 'TripA2 80': '빠름',
 'TripA3 74': '보통',
 'TripA3 75': '보통',
 'TripA3 76': '보통',
 'TripA3 77': '빠름',
 'TripA3 78': '빠름',
 'TripA3 79': '빠름',
 'TripA3 80': '빠름',
 'TripA3 81': '빠름',
 'TripA3 82': '보통',
 'TripA3 83': '보통',
 'TripA4 66': '보통',
 'TripA4 67': '빠름',
 'TripA4 68': '빠름',
 'TripA4 69': '빠름',
 'TripA4 70': '빠름',
 'TripA4 71': '보통',
 'TripA4 72': '빠름',
 'TripA4 73': '빠름',
 'TripA4 74': '빠름',
 'TripA4 75': '빠름',
 'TripA5 60': '좋음',
 'TripA5 61': '좋음',
 'TripA5 62': '좋음',
 'TripA5 63': '보통',
 'TripA5 64': '좋음',
 'TripA5 65': '좋음',
 'TripA5 66': '보통',
 'TripA6 63': '빠름',
 'TripA6 64': '좋음',


In [13]:
# TripB SoC 1% 감소 행 개수 빠름 보통 좋음 으로 분류
TripB_SoC_len_classification = {}

for j in range(38):
    globals()[j+1] = pd.read_csv('./NewTripB/TripB{}.csv'.format(j+1))
    startval = 0
    endval = 1
    # globals()['B_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            if globals()[i].shape[0] >= 150:
                TripB_SoC_len_classification ['TripB{}'.format(j+1)+' '+str(i)] = '좋음'
            elif globals()[i].shape[0] <50:
                TripB_SoC_len_classification ['TripB{}'.format(j+1)+' '+str(i)] = '빠름'
            else:
                TripB_SoC_len_classification ['TripB{}'.format(j+1)+' '+str(i)] = '보통'
        startval += 1
        endval += 1

In [70]:
TripB_SoC_len_classification

{'TripB1 57': '좋음',
 'TripB1 58': '보통',
 'TripB1 59': '보통',
 'TripB1 60': '빠름',
 'TripB1 61': '보통',
 'TripB1 62': '보통',
 'TripB1 63': '빠름',
 'TripB1 64': '보통',
 'TripB1 65': '보통',
 'TripB1 66': '좋음',
 'TripB1 67': '좋음',
 'TripB1 68': '좋음',
 'TripB1 69': '보통',
 'TripB1 70': '좋음',
 'TripB1 71': '보통',
 'TripB1 72': '보통',
 'TripB1 73': '빠름',
 'TripB1 74': '보통',
 'TripB1 75': '보통',
 'TripB1 76': '보통',
 'TripB1 77': '보통',
 'TripB1 78': '보통',
 'TripB1 79': '좋음',
 'TripB1 80': '좋음',
 'TripB1 81': '좋음',
 'TripB1 82': '보통',
 'TripB1 83': '보통',
 'TripB1 84': '보통',
 'TripB1 85': '보통',
 'TripB1 86': '빠름',
 'TripB2 66': '좋음',
 'TripB2 67': '보통',
 'TripB2 68': '빠름',
 'TripB2 69': '빠름',
 'TripB2 70': '보통',
 'TripB2 71': '보통',
 'TripB2 72': '보통',
 'TripB2 73': '보통',
 'TripB2 74': '좋음',
 'TripB2 75': '좋음',
 'TripB2 76': '좋음',
 'TripB2 77': '보통',
 'TripB2 78': '보통',
 'TripB2 79': '보통',
 'TripB2 80': '보통',
 'TripB2 81': '빠름',
 'TripB3 50': '보통',
 'TripB3 51': '보통',
 'TripB3 52': '빠름',
 'TripB3 53': '빠름',


In [79]:
#TripAsoc 폴더안에 있는 csv들을 soc행 계수를 time에, 나머지는 평균내어서 한줄로
for j in range(32):
    for i in range(100):
        if os.path.isfile(f'./TripAsoc/TripA{j+1}_soc{i}.csv') == True:
            df = pd.read_csv(f'./TripAsoc/TripA{j+1}_soc{i}.csv')
            dt = df.shape[0]
            df = df.describe().iloc[1:2]
            df.drop(['Time [s]'],axis=1,inplace=True)
            df.insert(0,'SoC_len',dt)
            df.to_csv(f'./TripAsoc_mean/TripA{j+1}_soc{i}_mean.csv',index=False)

In [158]:
#TripBsoc 폴더안에 있는 csv들을 soc행 계수를 time에, 나머지는 평균내어서 한줄로
for j in range(38):
    for i in range(100):
        if os.path.isfile(f'./TripBsoc/TripB{j+1}_soc{i}.csv') == True:
            df = pd.read_csv(f'./TripBsoc/TripB{j+1}_soc{i}.csv')
            dt = df.shape[0]
            df = df.describe().iloc[1:2]
            df.drop(['Time [s]'],axis=1,inplace=True)
            df.insert(0,'SoC_len',dt)
            df.to_csv(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv',index=False)

In [ ]:
#TripA 컬럼 맞추기
for j in range(32):
    for i in range(100):
        if os.path.isfile(f'./TripAsoc/TripA{j+1}_soc{i}.csv') == True:
            df = pd.read_csv(f'./TripAsoc/TripA{j+1}_soc{i}.csv')
            df.reindex(columns=col)

In [ ]:
#TripB 컬럼 맞추기
for j in range(38):
    for i in range(100):
        if os.path.isfile(f'./TripBsoc/TripB{j+1}_soc{i}.csv') == True:
            df = pd.read_csv(f'./TripBsoc/TripB{j+1}_soc{i}.csv')
            df.reindex(columns=col)

In [159]:
col = ['SoC_len', 'Velocity [km/h]', 'Elevation [m]', 'Throttle [%]',
       'Motor Torque [Nm]', 'Longitudinal Acceleration [m/s^2]',
       'Regenerative Braking Signal ', 'Battery Voltage [V]',
       'Battery Current [A]', 'Battery Temperature [캜]',
       'max. Battery Temperature [캜]', 'SoC [%]', 'displayed SoC [%]',
       'min. SoC [%]', 'max. SoC [%)', 'Heating Power CAN [kW]',
       'Heating Power LIN [W]', 'Requested Heating Power [W]',
       'AirCon Power [kW]', 'Heater Signal', 'Heater Voltage [V]',
       'Heater Current [A]', 'Ambient Temperature [캜]',
       'Ambient Temperature Sensor [캜]', 'Coolant Temperature Heatercore [캜]',
       'Requested Coolant Temperature [캜]', 'Coolant Temperature Inlet [캜]',
       'Coolant Volume Flow +500 [l/h]', 'Heat Exchanger Temperature [캜]',
       'Cabin Temperature Sensor [캜]', 'Temperature Coolant Heater Inlet [캜]',
       'Temperature Coolant Heater Outlet [캜]',
       'Temperature Heat Exchanger Outlet [캜]',
       'Temperature Defrost lateral left [캜]',
       'Temperature Defrost lateral right [캜]',
       'Temperature Defrost central [캜]',
       'Temperature Defrost central left [캜]',
       'Temperature Defrost central right [캜]',
       'Temperature Footweel Driver [캜]', 'Temperature Footweel Co-Driver [캜]',
       'Temperature Feetvent Co-Driver [캜]', 'Temperature Feetvent Driver [캜]',
       'Temperature Head Co-Driver [캜]', 'Temperature Head Driver [캜]',
       'Temperature Vent right [캜] ', 'Temperature Vent central right [캜]',
       'Temperature Vent central left [캜]', 'Temperature Vent right [캜]']

In [160]:
df = pd.read_csv('./concated_socmean/TripB_soc_con.csv')
df.reindex(columns = col)

,SoC_len,Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [캜],...,Temperature Footweel Driver [캜],Temperature Footweel Co-Driver [캜],Temperature Feetvent Co-Driver [캜],Temperature Feetvent Driver [캜],Temperature Head Co-Driver [캜],Temperature Head Driver [캜],Temperature Vent right [캜],Temperature Vent central right [캜],Temperature Vent central left [캜],Temperature Vent right [캜]
0,171,36.417220,476.828369,25.420480,3.044502,-0.165798,0.064327,372.562996,-7.076676,15.000000,...,24.080445,24.896989,37.598786,36.129859,22.483092,22.603410,29.138114,27.238113,28.906840,28.100897
1,138,44.173052,478.353676,19.457269,7.675730,-0.039940,0.101449,371.381229,-15.467709,15.084417,...,23.859720,25.176225,37.936111,35.930165,22.346502,22.755176,29.103428,27.663048,28.933536,28.027189
2,58,97.381943,478.894083,38.944556,9.384936,-0.142294,0.086207,367.351933,-38.298172,15.284350,...,23.965772,24.605594,37.390063,35.577201,21.879132,22.488846,29.346637,27.726129,28.275511,27.922059
3,7,106.792893,478.775370,97.526254,127.097999,1.915163,0.000000,336.385733,-336.027340,13.378569,...,24.236596,25.976209,37.456144,36.311639,22.461427,22.224517,29.618093,27.751593,28.690579,28.067140
4,70,60.596774,478.647164,38.812229,17.048816,0.067783,0.114286,370.913036,-30.916931,13.385714,...,24.148197,24.559572,37.063963,35.920969,22.422253,22.457951,29.213228,27.582206,28.357760,28.042462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,55,NaN,481.924727,38.793273,21.292364,-0.055091,0.000000,362.064182,-40.889091,0.000000,...,-0.767636,-0.905818,5.245273,4.390727,11.711818,13.739091,32.484182,32.257818,34.140182,31.527636
710,48,NaN,479.355000,39.717917,28.673542,0.056042,0.000000,360.733333,-47.680625,-0.188750,...,-0.752083,-0.982708,4.866875,5.744792,11.668333,13.732917,32.777708,30.425208,31.591042,30.543125
711,55,NaN,476.783455,32.722000,25.403455,0.119273,0.072727,364.048909,-38.526000,-1.000000,...,-1.088545,-0.993273,5.925455,11.744000,11.553273,14.186000,16.892545,16.372364,16.258727,15.213455
712,47,NaN,475.036596,35.903830,37.767872,0.156383,0.085106,362.230000,-48.404043,-1.000000,...,-1.423830,-0.909787,3.918511,8.544894,8.823191,11.688085,6.623404,8.996596,8.452128,7.807872


In [82]:
#저장한 파일들 개수 확인
a=[]
for j in range(32):
    for i in range(100):
        if os.path.isfile(f'./TripAsoc/TripA{j+1}_soc{i}.csv') == True:
            a.append(i)
print(len(a))

#저장한 파일들 개수 확인
a=[]
for j in range(38):
    for i in range(100):
        if os.path.isfile(f'./TripBsoc/TripB{j+1}_soc{i}.csv') == True:
            a.append(i)
print(len(a))

In [12]:
#빈 B용 csv파일 생성
df = pd.DataFrame(columns=['SoC_len', 'Velocity [km/h]', 'Elevation [m]', 'Throttle [%]',
       'Motor Torque [Nm]', 'Longitudinal Acceleration [m/s^2]',
       'Regenerative Braking Signal ', 'Battery Voltage [V]',
       'Battery Current [A]', 'Battery Temperature [캜]',
       'max. Battery Temperature [캜]', 'SoC [%]', 'displayed SoC [%]',
       'min. SoC [%]', 'max. SoC [%)', 'Heating Power CAN [kW]',
       'Heating Power LIN [W]', 'Requested Heating Power [W]',
       'AirCon Power [kW]', 'Heater Signal', 'Heater Voltage [V]',
       'Heater Current [A]', 'Ambient Temperature [캜]',
       'Ambient Temperature Sensor [캜]', 'Coolant Temperature Heatercore [캜]',
       'Requested Coolant Temperature [캜]', 'Coolant Temperature Inlet [캜]',
       'Coolant Volume Flow +500 [l/h]', 'Heat Exchanger Temperature [캜]',
       'Cabin Temperature Sensor [캜]', 'Temperature Coolant Heater Inlet [캜]',
       'Temperature Coolant Heater Outlet [캜]',
       'Temperature Heat Exchanger Outlet [캜]',
       'Temperature Defrost lateral left [캜]',
       'Temperature Defrost lateral right [캜]',
       'Temperature Defrost central [캜]',
       'Temperature Defrost central left [캜]',
       'Temperature Defrost central right [캜]',
       'Temperature Footweel Driver [캜]', 'Temperature Footweel Co-Driver [캜]',
       'Temperature Feetvent Co-Driver [캜]', 'Temperature Feetvent Driver [캜]',
       'Temperature Head Co-Driver [캜]', 'Temperature Head Driver [캜]',
       'Temperature Vent right [캜] ', 'Temperature Vent central right [캜]',
       'Temperature Vent central left [캜]', 'Temperature Vent right [캜]'])
df.to_csv('./concated_socmean/TripB_socmean_concated.csv',index=False)

In [13]:
#for문에 넣을 range선언
list_B = []
for j in range(38):
    listB=[]
    for i in range(100):
        if os.path.isfile(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv') == True:
            listB.append(i)
    min_val = min(listB)
    max_val = max(listB)
    range1 = range(min_val-1,max_val-2)
    list_B.append(range1)
list_B

#concat
df = pd.read_csv('./concated_socmean/TripB_socmean_concated.csv')
for j in range(38):
    # globals()[b[j]] = pd.read_csv(f'./TripBsoc_mean/TripB{j+1}_soc{b[j]}_mean.csv')
    for i in list_B[j]:
        if os.path.isfile(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv') == True:
            globals()[i] = pd.read_csv(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv')
            # globals()[i+1] = pd.read_csv(f'./TripBsoc_mean/TripB{j+1}_soc{i+1}_mean.csv')
            df = pd.concat([df,globals()[i]])
            #globals 변수 값을 다르게
    # df = pd.concat([df,globals()[c[j]]])
    # globals()[c[j+1]] = pd.concat([globals()[c[j]],globals()[c[j+1]]])

#중복 제거 후 저장
df.drop_duplicates(subset=['Temperature Defrost central [캜]','SoC [%]','Velocity [km/h]','Elevation [m]'])
df.to_csv('./concated_socmean/TripB_socmean_concatedB.csv',index=False)


In [ ]:
#동운이형 합치기

In [5]:
#TripA_soc_con 만들기
list_A = []
for i in range(32):
    globals()[i+1] = pd.read_csv(f'./NewTripA/TripA{i+1}.csv')
    min_val = int(globals()[i+1]['SoC [%]'].min())
    max_val = int(globals()[i+1]['SoC [%]'].max())
    range1 = range(min_val,max_val)
    list_A.append(range1)
list_A

clear1 = pd.DataFrame(columns=pd.read_csv('./TripAsoc_mean/TripA1_soc81_mean.csv').columns)
clear1.to_csv('./TripAsoc_mean/clear1.csv',index=False)

clear1 = pd.read_csv('./TripAsoc_mean/clear1.csv')

for idx,key in enumerate(list_A):
    for k in key:
        globals()[f'{idx},{k}'] = pd.read_csv(f'./TripAsoc_mean/TripA{idx+1}_soc{k}_mean.csv')
        clear1=pd.concat([clear1,globals()[f'{idx},{k}']])
clear1.to_csv('./new_concated_socmean/TripA_soc_con.csv',index=False)

In [3]:
#TripB_soc_con 만들기
list_B = []
for i in range(38):
    globals()[i+1] = pd.read_csv(f'./NewTripB/TripB{i+1}.csv')
    min_val = int(globals()[i+1]['SoC [%]'].min())
    max_val = int(globals()[i+1]['SoC [%]'].max())
    range2 = range(min_val,max_val)
    list_B.append(range2)
list_B

clear1 = pd.DataFrame(columns=pd.read_csv('./TripBsoc_mean/TripB1_soc57_mean.csv').columns)
clear1.to_csv('./TripBsoc_mean/clear1.csv',index=False)

clear1 = pd.read_csv('./TripBsoc_mean/clear1.csv')

for idx,key in enumerate(list_B):
    for k in key:
        globals()[f'{idx},{k}'] = pd.read_csv(f'./TripBsoc_mean/TripB{idx+1}_soc{k}_mean.csv')
        clear1=pd.concat([clear1,globals()[f'{idx},{k}']])
clear1.to_csv('./new_concated_socmean/TripB_soc_con.csv',index=False)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676 entries, 0 to 675
Data columns (total 50 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SoC_len                                676 non-null    int64  
 1   Unnamed: 0                             659 non-null    float64
 2   Velocity [km/h]                        659 non-null    float64
 3   Elevation [m]                          676 non-null    float64
 4   Throttle [%]                           676 non-null    float64
 5   Motor Torque [Nm]                      676 non-null    float64
 6   Longitudinal Acceleration [m/s^2]      676 non-null    float64
 7   Regenerative Braking Signal            676 non-null    float64
 8   Battery Voltage [V]                    676 non-null    float64
 9   Battery Current [A]                    676 non-null    float64
 10  Battery Temperature [캜]                676 non-null    float64
 11  max. B

In [11]:
df = pd.read_csv('./new_concated_socmean/TripB_soc_conb.csv')

#TripB_soc_con데이터 velocity [km/h]빈 부분에 velocity[km/h]]] 값 넣기
list(df.iloc[659:,48].values)

#velocity [km/h] 빈 부분에 값 넣기
df.iloc[659:,2] = [29.728405797101445,30.453308823529408,30.34453551912566,29.44192307692308,29.805066666666693,29.66121951219513,29.507129629629656,30.21635135135135,31.403456790123467,31.92669421487604,31.42567010309277,32.783181818181795,32.05777777777778,31.52763636363637,30.543125000000007,15.213454545454546,7.807872340425531]
#채워진거 확인
df

#필요없어진 컬럼 드롭하고
df.drop(columns=['Unnamed: 0','Velocity [km/h]]]'],inplace=True)
#저장
# df.to_csv('./concated_socmean/TripB_soc_con.csv',index=False)

#중복 없음
df.drop_duplicates(subset=['Temperature Defrost central [캜]','SoC [%]','Velocity [km/h]'],inplace=True)

#이상 없음
df.info()

#TripB_soc_con 컬럼 바꾸기, 결측치 채우기
# df = pd.read_csv('./concated_socmean/TripB_soc_conb.csv')
df.fillna(df.mean(),inplace=True)
df.columns = ['SoC_len',
 'Velocity [km/h]',
 'Elevation [m]',
 'Throttle [%]',
 'Motor Torque [Nm]',
 'Longitudinal Acceleration [m/s^2]',
 'Regenerative Braking Signal ',
 'Battery Voltage [V]',
 'Battery Current [A]',
 'Battery Temperature [°C]',
 'max. Battery Temperature [°C]',
 'SoC [%]',
 'displayed SoC [%]',
 'min. SoC [%]',
 'max. SoC [%)',
 'Heating Power CAN [kW]',
 'Heating Power LIN [W]',
 'Requested Heating Power [W]',
 'AirCon Power [kW]',
 'Heater Signal',
 'Heater Voltage [V]',
 'Heater Current [A]',
 'Ambient Temperature [°C]',
 'Ambient Temperature Sensor [°C]',
 'Coolant Temperature Heatercore [°C]',
 'Requested Coolant Temperature [°C]',
 'Coolant Temperature Inlet [°C]',
 'Coolant Volume Flow +500 [l/h]',
 'Heat Exchanger Temperature [°C]',
 'Cabin Temperature Sensor [°C]',
 'Temperature Coolant Heater Inlet [°C]',
 'Temperature Coolant Heater Outlet [°C]',
 'Temperature Heat Exchanger Outlet [°C]',
 'Temperature Defrost lateral left [°C]',
 'Temperature Defrost lateral right [°C]',
 'Temperature Defrost central [°C]',
 'Temperature Defrost central left [°C]',
 'Temperature Defrost central right [°C]',
 'Temperature Footweel Driver [°C]',
 'Temperature Footweel Co-Driver [°C]',
 'Temperature Feetvent Co-Driver [°C]',
 'Temperature Feetvent Driver [°C]',
 'Temperature Head Co-Driver [°C]',
 'Temperature Head Driver [°C]',
 'Temperature Vent right [°C] ',
 'Temperature Vent central right [°C]',
 'Temperature Vent central left [°C]',
 'Temperature Vent right [°C]']
df.to_csv('./new_concated_socmean/TripB_soc_con.csv',index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 676 entries, 0 to 675
Data columns (total 48 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SoC_len                                676 non-null    int64  
 1   Velocity [km/h]                        676 non-null    float64
 2   Elevation [m]                          676 non-null    float64
 3   Throttle [%]                           676 non-null    float64
 4   Motor Torque [Nm]                      676 non-null    float64
 5   Longitudinal Acceleration [m/s^2]      676 non-null    float64
 6   Regenerative Braking Signal            676 non-null    float64
 7   Battery Voltage [V]                    676 non-null    float64
 8   Battery Current [A]                    676 non-null    float64
 9   Battery Temperature [캜]                676 non-null    float64
 10  max. Battery Temperature [캜]           676 non-null    float64
 11  SoC [%

In [22]:
df = pd.read_csv('./concated_socmean/TripA_soc_con.csv')
df.iloc[:,15:16].fillna(0)
df.info()
list(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   SoC_len                              360 non-null    int64  
 1   Velocity [km/h]                      360 non-null    float64
 2   Elevation [m]                        360 non-null    float64
 3   Throttle [%]                         360 non-null    float64
 4   Motor Torque [Nm]                    360 non-null    float64
 5   Longitudinal Acceleration [m/s^2]    360 non-null    float64
 6   Regenerative Braking Signal          360 non-null    float64
 7   Battery Voltage [V]                  360 non-null    float64
 8   Battery Current [A]                  360 non-null    float64
 9   Battery Temperature [°C]             360 non-null    float64
 10  max. Battery Temperature [°C]        360 non-null    float64
 11  SoC [%]                         

['SoC_len',
 'Velocity [km/h]',
 'Elevation [m]',
 'Throttle [%]',
 'Motor Torque [Nm]',
 'Longitudinal Acceleration [m/s^2]',
 'Regenerative Braking Signal ',
 'Battery Voltage [V]',
 'Battery Current [A]',
 'Battery Temperature [°C]',
 'max. Battery Temperature [°C]',
 'SoC [%]',
 'displayed SoC [%]',
 'min. SoC [%]',
 'max. SoC [%]',
 'Heating Power CAN [kW]',
 'Heating Power LIN [W]',
 'Requested Heating Power [W]',
 'AirCon Power [kW]',
 'Heater Signal',
 'Heater Voltage [V]',
 'Heater Current [A]',
 'Ambient Temperature [°C]',
 'Coolant Temperature Heatercore [°C]',
 'Requested Coolant Temperature [°C]',
 'Coolant Temperature Inlet [°C]',
 'Heat Exchanger Temperature [°C]',
 'Cabin Temperature Sensor [°C]']

In [14]:
#TripA_soc_con 컬럼 바꾸기
df = pd.read_csv('./new_concated_socmean/TripA_soc_cona.csv')
df.drop(columns=['Heating Power LIN [W]','Heater Voltage [V]', 'Heater Current [A]','Coolant Temperature Heatercore [캜]','Coolant Temperature Inlet [캜]','Unnamed: 23'],inplace=True)
df.fillna(df.mean(),inplace=True)
df.columns = ['SoC_len',
 'Velocity [km/h]',
 'Elevation [m]',
 'Throttle [%]',
 'Motor Torque [Nm]',
 'Longitudinal Acceleration [m/s^2]',
 'Regenerative Braking Signal ',
 'Battery Voltage [V]',
 'Battery Current [A]',
 'Battery Temperature [°C]',
 'max. Battery Temperature [°C]',
 'SoC [%]',
 'displayed SoC [%]',
 'min. SoC [%]',
 'max. SoC [%]',
 'Heating Power CAN [kW]',
 'Requested Heating Power [W]',
 'AirCon Power [kW]',
 'Heater Signal',
 'Ambient Temperature [°C]',
 'Requested Coolant Temperature [°C]',
 'Heat Exchanger Temperature [°C]',
 'Cabin Temperature Sensor [°C]']
df.to_csv('./new_concated_socmean/TripA_soc_con.csv',index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   SoC_len                             328 non-null    int64  
 1   Velocity [km/h]                     328 non-null    float64
 2   Elevation [m]                       328 non-null    float64
 3   Throttle [%]                        328 non-null    float64
 4   Motor Torque [Nm]                   328 non-null    float64
 5   Longitudinal Acceleration [m/s^2]   328 non-null    float64
 6   Regenerative Braking Signal         328 non-null    float64
 7   Battery Voltage [V]                 328 non-null    float64
 8   Battery Current [A]                 328 non-null    float64
 9   Battery Temperature [°C]            328 non-null    float64
 10  max. Battery Temperature [°C]       328 non-null    float64
 11  SoC [%]                             328 non-n

In [11]:
#TripA,TripB_soc_mean 결측치들을 채워야함


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 28 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   SoC_len                              360 non-null    int64  
 1   Velocity [km/h]                      360 non-null    float64
 2   Elevation [m]                        360 non-null    float64
 3   Throttle [%]                         360 non-null    float64
 4   Motor Torque [Nm]                    360 non-null    float64
 5   Longitudinal Acceleration [m/s^2]    360 non-null    float64
 6   Regenerative Braking Signal          360 non-null    float64
 7   Battery Voltage [V]                  360 non-null    float64
 8   Battery Current [A]                  360 non-null    float64
 9   Battery Temperature [°C]             360 non-null    float64
 10  max. Battery Temperature [°C]        360 non-null    float64
 11  SoC [%]                         

In [ ]:
df = pd.read_csv('./concated_socmean/TripB_soc_con.csv')

df.drop(columns=['Unnamed: 0'],inplace=True)

#TripB_soc_con데이터 velocity [km/h]빈 부분에 velocity[km/h]]] 값 넣기
list(df.iloc[696:,48].values)

#velocity [km/h] 빈 부분에 값 넣기
df.iloc[696:,1] = [26.66724637681159,44.92838235294119,24.15142076502733,41.44107692307692,33.148600000000016,37.01886178861789,26.548842592592592,62.36513513513516,66.57185185185183,38.79099173553719,37.24278350515463,111.94477272727272,106.4388888888889,66.93309090909095,68.55125,39.94345454545454,34.75702127659574,6.18]

#채워진거 확인
df

#필요없어진 컬럼 드롭하고
df.drop(columns='Velocity [km/h]]]',inplace=True)
#저장
df.to_csv('./concated_socmean/TripB_soc_con.csv',index=False)

In [64]:
#TripA_soc_con 드랍컬럼
df = pd.read_csv('./concated_socmean/TripA_soc_con_before.csv')
df.drop(columns=['Unnamed: 23'],inplace=True)
df.to_csv('./concated_socmean/TripA_soc_con.csv')

In [ ]:
##
#TripAsoc 폴더안에 있는 csv들을 soc행 계수를 time에, 나머지는 평균내어서 한줄로..

#쓰레드 1초에 한번씩 갱신 그래프를 

#목금까지 쓰레드 1초에 한번씩 갱신 그래프를
#그 다음주는 데모까지 완성
# 수목금정도는 발표준비만

In [ ]:
#5번
#세트별로 모든 파라미터의 평균값을 내고
#좋은 세트일 때의 각 칼럼들의 평균 중 높은 것들이 뭐인지 보고
#나쁜 세트일 떄의 각 칼럼들의 평균 중 높은 것들이 뭐인지 보고


In [ ]:
#A세트 행 계수들 확인
for j in range(32):
    globals()[j+1] = pd.read_csv('./NewTripA/TripA{}.csv'.format(j+1))
    startval = 0
    endval = 1
    globals()['A_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            globals()['A_len_{}'.format(j+1)][i] = globals()[i].shape[0]
        startval += 1
        endval += 1

#B세트 행 계수들 확인
for j in range(38):
    globals()[j+1] = pd.read_csv('./NewTripB/TripB{}.csv'.format(j+1))
    startval = 0
    endval = 1
    globals()['B_len_{}'.format(j+1)] = {}
    for i in range(0,100):
        if len(globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)])>=1:
            globals()[i] = globals()[j+1][(globals()[j+1]['SoC [%]'] >= startval) & (globals()[j+1]['SoC [%]'] < endval)].reset_index(drop=True)
            globals()['B_len_{}'.format(j+1)][i] = globals()[i].shape[0]
        startval += 1
        endval += 1

In [8]:
##1줄모아 concat 실험코드

#TripB soc min길이 리스트
b=[]
for j in range(38):
    a=[]
    for i in range(100):
        if os.path.isfile(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv') == True:
            globals()[i+1] = pd.read_csv(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv')
            a.append(i)
    b.append(min(a))

#TripB soc max길이 리스트
c=[]
for j in range(38):
    d=[]
    for i in range(100):
        if os.path.isfile(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv') == True:
            globals()[i+1] = pd.read_csv(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv')
            d.append(i)
    c.append(max(d))

df = pd.read_csv('./concated_socmean/TripB_socmean_concated.csv')
for j in range(38):
    globals()[b[j]] = pd.read_csv(f'./TripBsoc_mean/TripB{j+1}_soc{b[j]}_mean.csv')
    for i in range(100):
        if os.path.isfile(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv') == True:
            globals()[i+1] = pd.read_csv(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv')
            globals()[i+1] = pd.concat([globals()[i],globals()[i+1]])
            #globals 변수 값을 다르게
    df = pd.concat([df,globals()[c[j]]])
    # globals()[c[j+1]] = pd.concat([globals()[c[j]],globals()[c[j+1]]])

In [34]:
for k in range(37):
    for j in range(38):
        for i in range(100):
            if os.path.isfile(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv') == True:
                globals()[i+1] = pd.read_csv(f'./TripBsoc_mean/TripB{j+1}_soc{i}_mean.csv')
        for l in range(100):
            try:
                globals()[l+2] = pd.merge([globals()[l+1],globals()[l+2]])
            except:
                pass
        globals()[j+1] = globals()[l+2]
    globals()[j+2] = pd.merge([globals()[j+1],globals()[j+2]])
globals()[j+2].to_csv('./concated_socmean/TripB_socmean_concated.csv',index=False)

KeyError: 101